# Group 1 - Predicting Airbnb Prices in Europe

Based on features like *room_type*, *location*, and *ratings*, can we predict Airbnb prices across major European cities?

## Dataset Description

***Kaggle Link:*** https://www.kaggle.com/datasets/thedevastator/airbnb-prices-in-european-cities/data?select=london_weekdays.csv

- ***weekend:***  1 if its a weekend and 0 if it is not

- ***host_is_superhost:***  if a host is a superhost, they meet these requirements (source: https://www.airbnb.com/help/article/829) :

    - Hosted at least 10 reservations, or 3 reservations that total at least 100 nights
    - Maintained a 90% or higher response rate
    - Maintained a less than 1% cancellation rate, with exceptions for cancellations due to Major Disruptive Events or other valid reasons
    - Maintained a 4.8 or higher overall rating (A review counts towards Superhost status when either both the guest and the host have submitted a review, or the 14-day window for reviews is over, whichever comes first.)

- ***bedrooms:***  in the case that bedrooms = 0, it might be because the listing is for a studio

- ***multiple_room:***  1 if listing is for multiple rooms, 0 otherwise.

- ***spare_room:***  if a listing is NOT for multiple rooms and is NOT for business purpose, then the listing is only for one room

- ***business_purpose:***  1 if it is for business purpose (full time income generating) or not (personal use, for example renting out a spare room for a few weeks). For this classification, we assume that business listings typically feature more rooms than the threshold for multiple rooms column. This is because we noticed that there are listings that are NOT multiple rooms and NOT for business purpose.

- ***price:*** we took the natural log of the price because we noticed that the price is very skewed.

In [ ]:
# Data Preprocessing
import pandas as pd
import numpy as np
import zipfile
import os

# Exploratory Data Analysis
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

# Principal Component Analysis
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import decomposition

import warnings
warnings.filterwarnings("ignore")

: 

## Data Preprocessing

Let's preprocess the datasets in *Airbnb_data.zip* to understand the features and how to proceed with Exploratory Data Analysis (EDA).

In [ ]:
# # Load Dataset - COLAB
# from google.colab import files
# uploaded = files.upload()
# !unzip Airbnb_data.zip

# datasets = {}
# for _, _, filenames in os.walk("Airbnb_data"):
#   for file_name in filenames:
#     name = file_name.split(".")[0]
#     datasets[name] = pd.read_csv("Airbnb_data/" + file_name)
# datasets

: 

In [ ]:
# Load Dataset - VSCode
path = "Airbnb_data.zip"
folder = "Airbnb_data"

# extract zipfile
with zipfile.ZipFile(path, "r") as zip_ref:
    zip_ref.extractall(folder)

# list extracted files
files = os.listdir(folder)
print(f"Extracted Files: {files}\n")

# load csv files
datasets = {}

for file in files:
    if file.endswith(".csv"):
        print(f"Loading file: {file}")
        path = os.path.join(folder, file)
        df = pd.read_csv(path)
        datasets[file.rsplit(".", 1)[0]] = df # remove ".csv"

: 

Now that we've loaded the airbnb datasets, we can continue with data preprocessing to prepare it for EDA.

### i) Add "City" & "Weekend" Columns

In [ ]:
# Add "city" & "weekend" Columns
for name, dataset in datasets.items():
    city, weekend = name.split("_")
    dataset["city"] = city
    dataset["weekend"] = weekend == "weekends"
    dataset["weekend"] = dataset["weekend"].astype(int)

: 

### ii) Combine Datasets

In [ ]:
# Combine Datasets
df = pd.concat([datasets[file_name] for file_name in datasets])
df.head()

: 

***Observation:*** Based on the concatenated airbnb dataset, we see that the *unnamed* column is simply the index from the original dataset. We don't need this anymore...

### iii) Clean Data

First, let's standardize the column names for better readability and consistency.

In [ ]:
# Standardize Column Names
name = {'realSum' :'price',
        'biz': 'business_purpose',
        'multi' : 'multiple_rooms',
        'dist' : 'dist_city_center',
        'metro_dist' : 'dist_metro_station',
        'lng' : 'longitude',
        'lat':'latitude'
        }

df = df.rename(columns = name)
df.head()

: 

***Observation:*** The *Unnamed* column is redundant and unecessary as it's the same as the original dataset's indices.

So let's drop *Unnamed* from the dataset.

In [ ]:
# Drop "Unnamed" Column
df = df.drop(columns =['Unnamed: 0'])
df.head()

: 

Now let's look at the descriptive statistics of our new dataset. This will give us insight into skewness and outliers in the dataset that we will need to handle.

In [ ]:
# Descriptive Statistics
df.describe()

: 

***Observation:*** Based on the descriptive statistics, *price* is heavily skewed.

Let's use log to transform the *price* column so it's standardized.

In [ ]:
# Standardize "price" Column
df["price"] = np.log(df['price'])
df.describe()

: 

Let's add a *spare_room* column to the dataset based on the *multiple_rooms* and *business_purpose* columns.

This will add more insight into the types of listings users are booking.

In [ ]:
# Add "spare_room" Column
df['spare_room'] = ((df['multiple_rooms'] == 0) & (df['business_purpose'] == 0))
df.head()

: 

Now that we handled inconsistent data, we can handle missing data.

Let's see if and where our dataset has missing values so we can determine whether to drop or imputate it.

In [ ]:
# Check Missing Data
np.sum(df.isnull(), axis = 0)

: 

***Observation:*** Our dataset has no null values! So there's no need to handle missing data and we can continue with our EDA.

## Exploratory Data Analysis (EDA)

Next, let's take an initial look at the features in our dataset, so we can gain more insight about their roles and characteristics.

### i) Feature Distributions

In [ ]:
# Graph Functions
def init_subplots(rows, cols, figsize, df):
    fig, axes = plt.subplots(rows, cols, figsize=figsize) # init grid side
    axes = axes.flatten()
    cmap = cm.get_cmap('inferno', len(df.columns)) # inferno colors
    return fig, axes, cmap

def separate_features(df):
    numerical_cols = [col for col in df.columns if df[col].dtype == "float64"]
    categorical_cols = [col for col in df.columns if df[col].dtype != "float64"]
    all_columns = numerical_cols + categorical_cols
    return numerical_cols, categorical_cols, all_columns

def color_palette(cmap, idx, df, col):
    color = cmap(idx / len(df.columns))                 # create colormap
    unique_vals = df[col].nunique()                     # get unique vals
    palette = sns.color_palette("inferno", unique_vals) # map palette to unique vals
    return color, palette

def show_plots(df, fig, axes, layout):
    for i in range(len(df.columns) - 1, len(axes)): # hide unused subplots
        fig.delaxes(axes[i])

    if layout != None: fig.tight_layout(rect=layout) # setup layout
    else: fig.tight_layout()
    fig.show()
    return

: 

In [ ]:
# Initialize Plots
numerical_cols, categorical_cols, all_columns = separate_features(df) # separate types
num_numerical = len(numerical_cols)
num_categorical = len(categorical_cols)

: 

Let's plot the distributions for *numerical features* in the dataset using *histograms*.

In [ ]:
# Histogram Distributions - Numerical Features
fig, axes, cmap = init_subplots(4, 3, (10,12), df) # init subplot

for idx, col in enumerate(numerical_cols):
    ax = axes[idx]                               # get subplot idx
    color, _ = color_palette(cmap, idx, df, col) # init color
    sns.histplot(df, x=col, ax=ax, color=color)
    ax.set_title(col)
    ax.set_xlabel(col, fontsize=8)
    ax.set_ylabel("count", fontsize=8)

fig.text(0.5, 0.985, "Numerical Features - Histogram Distributions", fontsize=16, ha="center")
show_plots(df, fig, axes, [0, 0, 1, 0.97]) # display plots

: 

**Observations - Numerical**

Based on the *histogram* distributions, we can see that...

- ***Price:*** Heavily skewed to the right, indicating that most listing have lower prices with some high-priced outliers.

- ***Guest Satisfaction:*** Skewed toward high satisfaction, with many ratings clustered around 100.

- ***Distance:*** Most listings are within 5km of the city center.

- ***Restaurants & Attractions:*** Heavily skewed to the right, indicating that most listings are near fewer restaurants/attractions, while a small number are near a high density of them.

- ***Location:*** Based on *longitude* and *latitude*, listings are clearly concentrated in specific geographic regions as seen by the clustering.

Now let's plot the distributions for *categorical features* in the dataset using *countplots*.

In [ ]:
# Countplot Distributions - Categorical Features
fig, axes, cmap = init_subplots(4, 3, (10,12), df) # init subplot

for idx, col in enumerate(categorical_cols):
    ax = axes[idx]                                             # get subplot idx
    _, palette = color_palette(cmap, num_categorical, df, col) # init palette
    sns.countplot(df, x=col, ax=ax, palette=palette, hue=col, legend=False)
    ax.set_title(col)
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), rotation=60, ha="center")

for ax in axes[len(categorical_cols):]: ax.remove()
fig.text(0.5, 1.02, "Categorical Features - Countplot Distributions", fontsize=16, ha="center")
show_plots(df, fig, axes, None) # display plots

: 

**Observations - Categorical**

Based on the *countplot* distributions, we can see that...

- ***Room Types:*** "Entire home/apt" is the majority of the listings, followed by "Private room" then "Shared room".

- ***Room Features:*** A significant number of listings have private rooms where its capacity is around 2-4 people.

- ***Superhost Status:*** Majority of hosts are not superhosts, suggesting that becoming a superhost is rare.

- ***City Distribution:*** Majority of listsings in London, followed by Rome and Paris. There are significant variations in the number of listings across the other cities.

- ***Weekend:*** Most listings are not flagged as "weekend", which might indicate mostly weekday stays or a lack of weekend-specific data.

### ii) Correlation Analysis

First, let's plot the correlation distributions for *numerical features* in the dataset using *scatterplots*.

In [ ]:
# Scatterplot Distributions - Numerical Features
fig, axes, cmap = init_subplots(4, 3, (10, 12), df) # init subplots
numerical_cols, _, _ = separate_features(df)        # separate types

for idx, col in enumerate(numerical_cols):
    if col == "price":  # skip price column
        continue

    else:
        ax = axes[idx]  # Get subplot
        color, _ = color_palette(cmap, idx, df, col)  # Init color
        sns.scatterplot(x=df[col], y=df["price"], ax=ax, color=color)
        ax.set_title(f"price vs {col}")
        ax.set_xlabel(col, fontsize=8)
        ax.set_ylabel("price", fontsize=8)

fig.text(0.5, 1.02, "Price Correlation Pairplots - Numerical Features", fontsize=16, ha="center")
show_plots(df, fig, axes, None)  # Display numerical pairplots

: 

**Observations - Numerical:**

In relation to *price*, the following numerical features are...

- ***person_capacity:*** positively correlated

- ***guest_satisfaction_overall:*** positively correlated

- ***attr_index:*** positively correlated

- ***rest_index:*** positively correlated

- ***dist_city_center:*** negatively correlated

- ***location:*** not significantly correlated

Now let's plot the correlation distributions for *categorical features* in the dataset using *boxplots*.

In [ ]:
# Boxplot Distributions - Categorical Features
fig, axes, cmap = init_subplots(4, 3, (10, 12), df) # init subplots
_, categorical_cols, _ = separate_features(df)      # separate types

for idx, col in enumerate(categorical_cols):
    ax = axes[idx]  # Get subplot
    _, palette = color_palette(cmap, idx, df, col)  # init palette
    sns.boxplot(x=df[col], y=df["price"], ax=ax, palette=palette)
    ax.set_title(f"price vs {col}")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=60, ha="center")
    ax.set_xlabel(col, fontsize=8)
    ax.set_ylabel("price", fontsize=8)

for ax in axes[len(categorical_cols):]: ax.remove()
fig.text(0.5, 1.02, "Price Correlation Boxplots - Categorical Features", fontsize=16, ha="center")
show_plots(df, fig, axes, None)  # display plots

: 

**Observations - Categorical:**

In relation to *price*, the following categorical features are...

- ***room_type:*** positively correlated

- ***host_is_superhost:*** positively correlated but weak

- ***multiple_rooms:*** positively correlated

- ***business_purpose:*** positively correlated

- ***city:*** not linearly correlated

- ***weekend:*** not correlated

## Principal Component Analysis (PCA)

Let's use *PCA Clustering* to gain more insight about the structure, relationships, and patterns in our dataset.

### i) Train, Test Split


In [ ]:
# Train, Test, Split
X = df.drop(columns = ['price']) # drop "price" col
y = df['price']

# split into train, test, validation sets
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size = 0.15, random_state = 1234, shuffle = True)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.17647, random_state = 1234, shuffle = True)

: 

### ii) Standardize Data

In [ ]:
# Get Data
X_df = X_train
X_df = X_df.drop(columns = ['room_type']) # drop bc already encoded
X_df = pd.get_dummies(X_df)
X_df.head()

: 

In [ ]:
# Standardize Data
scaler = preprocessing.StandardScaler()
scaled_X_train_df = scaler.fit_transform(X_df)

# standardize X_test and X_valid
scaled_X_valid_df = X_valid.drop(columns = ['room_type'])
scaled_X_valid_df = pd.get_dummies(scaled_X_valid_df)
scaled_X_valid_df = scaler.transform(scaled_X_valid_df)

scaled_X_test_df = X_test.drop(columns = ['room_type'])
scaled_X_test_df = pd.get_dummies(scaled_X_test_df)
scaled_X_test_df = scaler.transform(scaled_X_test_df)

: 

### iii) Conduct PCA

In [ ]:
# Compute Explained Variance
pca = decomposition.PCA(random_state = 1234).fit(scaled_X_train_df)                  # perform PCA
explained_var = round(sum(pca.explained_variance_ratio_[:13]), 4) # compute explained variance
print(f"Explained Variance = {explained_var}")

: 

***Observation:*** Based on the explained variance computer above, the first 13 principal components explain around 79% of the variability in the data. This suggests that, after PCA, the reduced dimensions are still highly representative of the original data.

Let's transform the *training* data.

Note: We only want to keep the first 13 principal components because our model is able to explain around 79% of the variance in the data.

In [ ]:
# Transformation - Training Data
scaled_X_train_df = pca.transform(scaled_X_train_df)
scaled_X_train_df = pd.DataFrame(scaled_X_train_df)
scaled_X_train_df.columns = ['PC_' + str(i) for i in range(1, 30)]
scaled_X_train_df = scaled_X_train_df.iloc[:,0:13]
scaled_X_train_df.head()

: 

Let's take a look at the *eigenvectors* in the dataset.

They will help us determine how much each original feature contributes (weighs) to a principal component.

In [ ]:
# Get Eigenvectors
eigenvectors = pd.DataFrame(pca.components_, columns =['PC_' + str(i) for i in range(1, 30)])
eigenvectors = eigenvectors.iloc[:, 0:13]
eigenvectors.head()

: 

Now let's visualize the eigenvectors for each principal component. More specifically, let's look at how much of each feature contributed to the corresponding principal component.

In [ ]:
# Eigenvector Barplots
fig, axes, cmap = init_subplots(8, 3, (10,20), df) # init plots

for idx, PC in enumerate(eigenvectors.columns):
    ax = axes[idx] # get subplot idx
    palette = sns.color_palette("inferno", n_colors=len(eigenvectors.index))

    sns.barplot(x=eigenvectors.index, y=eigenvectors[PC], ax=ax, palette=palette)
    ax.set_title(PC)
    ax.set_xlabel("OG Features", fontsize=8)
    ax.set_ylabel("Contribution", fontsize=8)

    # setup x-ticks parameters
    xticks = ax.get_xticks()
    ax.set_xticks(xticks[::5]) # skip every 5 ticks
    ax.tick_params(axis='x')

fig.suptitle("Eigenvector Contributions", fontsize=14, y=1)
show_plots(eigenvectors, fig, axes, None) # display plots

: 

***Observations:*** Based on these principal component barplots, many of the components have a single feature that has a significant, positive contribution. This suggests that those features contribute the most in terms of explaining the variance captured in that given principal component.

Now let's transform the *validation* and *test* data too so we can visualize the PCA results.

In [ ]:
# Transformation - Valid Data
scaled_X_valid_df = pca.transform(scaled_X_valid_df)
scaled_X_valid_df = pd.DataFrame(scaled_X_valid_df)
scaled_X_valid_df.columns = ['PC_' + str(i) for i in range(1, 30)]
scaled_X_valid_df = scaled_X_valid_df.iloc[:,0:13]
scaled_X_valid_df.head()

: 

In [ ]:
# Transformation - Test Data
scaled_X_test_df = pca.transform(scaled_X_test_df)
scaled_X_test_df = pd.DataFrame(scaled_X_test_df)
scaled_X_test_df.columns = ['PC_' + str(i) for i in range(1, 30)]
scaled_X_test_df = scaled_X_test_df.iloc[:,0:13]
scaled_X_test_df.head()

: 

In [ ]:
# PCA Scatterplot
plt.figure(figsize=(8, 6))
plt.scatter(scaled_X_valid_df["PC_1"], scaled_X_valid_df["PC_2"], alpha=0.6, label="Validation Data", color="#84206b")
plt.scatter(scaled_X_test_df["PC_1"], scaled_X_test_df["PC_2"], alpha=0.6, label="Test Data", color="#e55c30")
plt.title("PCA Results: Validation and Test Data")
plt.legend()
plt.xlabel("PC_1")
plt.ylabel("PC_2")
plt.grid(True)
plt.show()

: 

**Observations**

There's a significant overlap between *validation* and *test* datasets in the PCA space. This suggests...

- ***Consistency:*** The feature representation across these datasets is consistent

- ***Simplicity:*** Without distinct clustering and separation, the dataset doesn't show strong non-linear patterns

- ***Generalizability:*** Components can effectively represent unseen data (validation and test sets)

Based on the *PCA*, the identified principal components capture the variance in the dataset while reducing multicolinearity among features.

## Method I - Multi-Linear Regression (MLR)

For our first method, let's use *Multi-Linear Regression (MLR)* to build a model for predicting Airbnb prices while mitigating overfitting.

This model will help us to uncover the relationship between a *dependent* variable (price) and multiple *independent variables*.

### i) Initialize Training & Testing Variables

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

: 

In [ ]:
# Initialize Variables (X_train)
X_train = pd.get_dummies(X_train, columns=["room_type", "city"], drop_first=True)
X_train_np = X_train.values.astype(float)
X_train_np = np.hstack((X_train_np, np.ones((X_train_np.shape[0], 1)))) # add bias
y_train_np = y_train.values.astype(float)

# Initialize Variables (X_test)
X_test = pd.get_dummies(X_test, columns=["room_type", "city"], drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
X_test_np = X_test.values.astype(float)
X_test_np = np.hstack((X_test_np, np.ones((X_test_np.shape[0], 1)))) # add bias

: 

### ii) Conduct Model

In [ ]:
# Multi-Linear Regression
MLR_model = LinearRegression()
MLR_model.fit(X_train_np, y_train_np) # fit

: 

### iii) Evaluate Model

In [ ]:
# Functions for Evaluation Metrics
def adjusted_r2(y_test, X_train_np, r2):
    n = len(y_test)          # num observations
    k = X_train_np.shape[1]  # num predictors
    MLR_adjusted_r2 = 1 - ((1 - r2) * (n - 1)/(n - k - 1)) # adjusted R^2
    return MLR_adjusted_r2

def eval_metrics(mse, r2, adjusted_r2):
    print(f"Test MSE:    {mse:.3f}")
    print(f"Test R²:     {r2:.4f}")
    print(f"Adjusted R²: {adjusted_r2:.4f}")
    return

: 

In [ ]:
# Evaluate MLR Model
y_pred = MLR_model.predict(X_test_np)                     # prediction
MLR_mse = mean_squared_error(y_test.values, y_pred)       # mean-squared-error
MLR_r2 = r2_score(y_test.values, y_pred)                  # unadjusted R^2
MLR_adjusted_r2 = adjusted_r2(y_test, X_train_np, MLR_r2) # adjusted R^2
eval_metrics(MLR_mse, MLR_r2, MLR_adjusted_r2)            # print metrics

: 

***Observations:*** These evaluation metric scores suggest that our model performs *reasonably well*. Since ~34% of the variability is not explained, this is likely due to missing features, randomness, and noise in our data.

## Analysis

### i) Numerical Comparison

In [ ]:
# Compare Actual vs Predicted Prices
y_scaler = preprocessing.StandardScaler()
y_scaler.fit(y_train.values.reshape(-1, 1)) # fit on y_train data

# map city columns
city_columns = [col for col in X_test.columns if col.startswith("city_")] # find city cols
city_map = {col: col.split("_")[1] for col in city_columns}               # map col to city names
X_test["city"] = X_test[city_columns].idxmax(axis=1).map(city_map)        # find city name for each row

# create comparison_df
comparison_df = pd.DataFrame({
    "City" : X_test["city"],        # get city
    "Actual Price" : y_test.values, # actual prices
    "Predicted Price" : y_pred      # predicted prices
})

# inverse transformations on prices
comparison_df["Actual Price"] = y_scaler.inverse_transform(comparison_df[["Actual Price"]].values)
comparison_df["Actual Price"] = np.exp(comparison_df["Actual Price"])

comparison_df["Predicted Price"] = y_scaler.inverse_transform(comparison_df[["Predicted Price"]].values)
comparison_df["Predicted Price"] = np.exp(comparison_df["Predicted Price"])

# group by city and compute avg prices
comparison_grouped = comparison_df.groupby("City").mean()
comparison_grouped = comparison_grouped.applymap(lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else x)
comparison_grouped.reset_index(inplace=True)

print("  Actual vs Predicted Airbnb Prices ($)")
comparison_sorted = comparison_grouped.sort_values(by="Actual Price", ascending=True) # sort prices in ascending order
comparison_sorted

: 

In the table above are the average *average* and *predicted* prices of Airbnb listings per European city.

- ***Pros:*** The *predicted* prices align closely with the *actual* prices across all cities, indicating that our model performs well overall.

- ***Cons:*** There are slight variations, like in *London* and *Paris*, where the *predicted* values slightly underestimate the *actual* prices.

### ii) Graphical Comparison

Second, let's create several plots to visually analyze how our *MLR* model performed in predicting Airbnb prices.

Let's look at a *scatterplot*.

In [ ]:
# Function - Plot Actual vs Predicted Prices
def comparison_scatterplot(y_test, y_pred, filename=None):
    min_price = min(y_test.min(), y_pred.min()) # compute min
    max_price = max(y_test.max(), y_pred.max()) # compute max

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(y_test, y_pred, alpha=0.7, color="#84206b", label="City Data")
    ax.plot([min_price, max_price], [min_price, max_price], "r--", color="#e55c30", label="Ideal Line")
    ax.set_title("Actual vs Predicted Airbnb Prices")
    ax.set_xlabel("Actual")
    ax.set_ylabel("Predicted")
    ax.legend()

    if filename: fig.savefig(filename) # save figure for later

    return fig.show(), fig

: 

In [ ]:
# Scatterplot
MLR_scatterplot = comparison_scatterplot(y_test, y_pred, filename="MLR_scatterplot.png")
MLR_scatterplot

: 

Based on our *scatterplot* of actual vs predicted prices, our model...

- ***Performance:*** performs reasonably well given many predictions align closely with the *ideal line*

- ***Errors:*** contains some outliers that indicate areas where our model's predictions deviate more significantly from *actual* prices

Now let's look at a *barplot*.

In [ ]:
# Barcharts
comparison_sorted["Actual Price"] = comparison_sorted["Actual Price"].replace(',', '', regex=True).astype(float)
comparison_sorted["Predicted Price"] = comparison_sorted["Predicted Price"].replace(',', '', regex=True).astype(float)

# extract sorted values
cities = comparison_sorted["City"].values                      # city names
actual_prices = comparison_sorted["Actual Price"].values       # actual prices
predicted_prices = comparison_sorted["Predicted Price"].values # predicted prices
position = np.arange(len(cities))                              # x-axis position

# combine actual/predicted prices
all_prices = np.concatenate((actual_prices, predicted_prices))
min_price = np.floor(all_prices.min() / 500) * 500  # round to neareast 500
max_price = np.ceil(all_prices.max() / 500) * 500

# plot
plt.figure(figsize=(6, 4))
plt.bar(position - 0.2, actual_prices, width=0.4, label="Actual Price", color="#84206b")
plt.bar(position + 0.2, predicted_prices, width=0.4, label="Predicted Price", color="#e55c30")
plt.title("Actual vs Predicted Airbnb Prices by City")
plt.xlabel("City")
plt.ylabel("Price ($)")
plt.xticks(position, cities, rotation=45)     # city names
plt.ylim(min_price, max_price)                # ensure y-axis range matches data
plt.grid(axis="y", linestyle="--", alpha=0.7) # grid for y-axis only
plt.tight_layout()
plt.legend()
plt.show()

: 

Based on our *barplot* of actual vs predicted prices by city, our model...

- ***Predictions:*** The *actual* Airbnb prices tend to be higher than our *predicted* prices for all cities.

- ***Highest Prices:*** Paris has the highest Airbnb prices, followed by London and Barcelona.

- ***Lowest Prices:*** Budapest has the lowest Airbnb prices, followed by Rome then Lisbon.

### Conclusion

We conclude that our *Multi-Linear Regression* model...

- Demonstrates a reasonable ability to predict Airbnb prices across various European cities.

- Contains certain city-specific factors may require additional consideration to enhance prediction accuracy.

- Provides insights that can help property owners and managers develop data-driven pricing strategies that reflect both competitive standards and localized dynamics.

***Suggestions:*** Because we can improve our model's performance, let's try more complex models like *Gradient Boosting* and *Deep Neural Networks* in order to more accurately predict Airbnb prices.These models will help us to capture complex, non-linear relationships between multiple features and *price*.

## Method II - Gradient Boosting

For our second method, let's use *Gradient Boosting* which will help us to...

- Handle outliers
- Get feature interactions
- Compute feature importance analysis

### i) Prepare Data

In [ ]:
# Import Packages
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

: 

In [ ]:
# One-Hot Encoding
df["room_shared"] = df["room_shared"].apply(lambda x: 1 if x else 0)
df["room_private"] = df["room_private"].apply(lambda x: 1 if x else 0)
df["host_is_superhost"] = df["host_is_superhost"].apply(lambda x: 1 if x else 0)
df["spare_room"] = df["spare_room"].apply(lambda x: 1 if x else 0)

df = pd.get_dummies(df, columns=["room_type"], drop_first=False)
df.head()

: 

### ii) Conduct Model

In [ ]:
# Run Model
X = df.drop(columns=["price", "city"]) # drop target/non-predictive cols
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)               # train
GB_X_train = X_train # need for later
GB_model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=42) # build model
GB_model.fit(X_train, y_train)

: 

### iii) Evaluate Model

In [ ]:
# Evaluate GB Model
y_pred = GB_model.predict(X_test)                       # make predictions
GB_mse = mean_squared_error(y_test, y_pred)             # compute MSE
GB_r2 = r2_score(y_test, y_pred)                        # compute R^2
GB_adjusted_r2 = adjusted_r2(y_test, X_train_np, GB_r2) # adjusted R^2
eval_metrics(GB_mse, GB_r2, GB_adjusted_r2)             # print metrics

: 

***Observations:*** These evaluation metric scores suggest that our model performs *well*! Since ~24% of the variability is not explained, our model's performance has improved from the *MLR* model.

## Analysis

Let's graphically visualize our results with a comparative *scatterplot*.

In [ ]:
# Scatterplot
GB_scatterplot = comparison_scatterplot(y_test, y_pred, filename="GB_scatterplot.png")
GB_scatterplot

: 

***Observations:*** Compared to our *MLR* model, the *Gradient Booster* model shows a tighter clustering of points along the ideal line, which indicates a better alignment between *predicted* and *actual* prices.

### Conclusion

Unlike our *Multi-Linear Regression* model, this model...

- ***Handles Non-Linearity:*** better captures complex, non-linear relationships between features *(room_type, dist_city)* and *price*

- ***Feature Interactions:*** identifies and models interactions between features like *[room_type, number_of_guests] or [city, availability]*, which leads to improved predictions

- ***Minimized Errors:*** decreased *MSE* in most cities, especially for cities with more variability *(Vienna and Budapest)*

## Method III - Deep Neural Networks

For our third method, let's use *Deep Neural Networks* which will help us to...

- Improve predictive accuracy on large datasets
- Explore advanced feature representations

### i) Prepare Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_absolute_error

: 

In [ ]:

# Seperate target and feature variables
X = df.drop('price', axis=1)
y = df['price']

# Encode categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
X_encoded = pd.get_dummies(X, columns=categorical_columns)

# Scale numerical columns
scaler = StandardScaler()
X_scaled = X_encoded.copy()
X_scaled[numerical_columns] = scaler.fit_transform(X_scaled[numerical_columns])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.17647, random_state=42)

: 

### ii) Conduct Model

In [ ]:
# Define the model with tuned hyperparameters
def build_model(input_shape):
    model = keras.Sequential([
        layers.Dense(288, activation='relu', input_shape=[input_shape]),
        layers.Dense(320, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])
    optimizer = keras.optimizers.Adam(learning_rate=0.0002)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Train
input_shape = X_train.shape[1]
model = build_model(input_shape)
history = model.fit(
    X_train, y_train,
    validation_split=0.17647,
    epochs=100,
    batch_size=32,
    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=1
)

: 

### iii) Evaluate Model

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.title('Model Training History')
plt.legend()
plt.show()

: 

**Observations**

- ***Initial Convergence:*** training loss has a steep decline and stabilizes quickly, indicating that the model learns effectively
- ***After Convergence:*** both losses reach a pleteau with minimal fluctuations, which suggests that the model achieved a stable state

In [ ]:
# Compute Evaluation Metrics
DNN_mae = mean_absolute_error(y_test, y_pred)             # mean-absolute error
DNN_mse = mean_squared_error(y_test, y_pred)              # mean-squared error
DNN_rmse = np.sqrt(DNN_mse)                               # root-mean-squared error
DNN_r2 = r2_score(y_test, y_pred)                         # R^2
DNN_adjusted_r2 = adjusted_r2(y_test, X_train_np, DNN_r2) # adjusted R^2
eval_metrics(DNN_mse, DNN_r2, DNN_adjusted_r2)            # print metrics

: 

***Observations:*** Based on these evaluation metrics scores, our *Deep Neural Networks* model performs well and indicates a good fit.

### Analysis

Let's graphically visualize our results with a *scatterplot*.

In [ ]:
# Scatterplot
DNN_scatterplot = comparison_scatterplot(y_test, y_pred, filename="DNN_scatterplot.png")
DNN_scatterplot

: 

***Observations:*** Compared to our *Gradient Booster* model, the *Deep Neural Network* model shows more dispersed clustering of points along the ideal line, which indicates a slightly worse alignment between *predicted* and *actual* prices.

Now let's plot our residuals from this model.

In [ ]:
# Residuals Scatterplot
residuals = y_test - y_pred.flatten() # flatten

plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, alpha=0.5, color="#84206b")
plt.axhline(y=0, linestyle='--', color="#e55c30")
plt.title('Residuals Plot')
plt.xlabel('Predicted Price')
plt.ylabel('Residuals')
plt.show()

: 

***Observations:*** The residuals are distributed around the zero line, indicating that the model has minimal bias in its predictions. However, there are a few significant outliers where the residuals are far from zero.

So let's try to optimize our *Deep Neural Networks* model with Tensorflow's *keras_tuner* package.

### i) Build Model

In [ ]:
# Run Optimal DNN Model
import keras_tuner as kt

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=hp.Int('units_1', min_value=32, max_value=512, step=32), activation='relu', input_shape=(X_train.shape[1],)))

    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(keras.layers.Dense(units=hp.Int(f'units_{i+2}', min_value=32, max_value=512, step=32), activation='relu'))

    model.add(keras.layers.Dense(1))
    model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')), loss='mean_squared_error')
    return model

: 

### ii) Tune Hyperparameters

In [ ]:
# Tune Hyperparameters
tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=100,
    factor=3,
    directory='my_dir',
    project_name='airbnb_price_prediction')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("The hyperparameter search is complete.\n")
print(f"Optimal # of Layers:   {best_hps.get('num_layers')}")
print(f"Optimal Learning Rate: {best_hps.get('learning_rate'):.4f}")

: 

### iii) Evaluate Optimal Model

In [ ]:
# Evaluate Optimal Model
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2) # fit model

test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

: 

### Conclusion

We conclude that our *Neural Network* model...

- ***Performance:*** demonstrates a reasonable ability to *predict* Airbnb prices based on the accuracy and R^2

- ***Suggestions:***
    - Further tune parameters for optimal prediction
    - Potentially overfits the data due to the validation loss converging with training loss and then begins to increase. Could be a result of containing such a large sum of data

## Executive Summary

### i) Compare Model Performances

Let's compare our *Multi-Linear Regression*, *Gradient Booster*, and *Deep Neural Networks* models to identify which performed the best at predicting Airbnb prices.

In [ ]:
# Compare Results
saved_plots = [
    ("Multi-Linear Regression", "MLR_scatterplot.png"),
    ("Gradient Boosting", "GB_scatterplot.png"),
    ("Deep Neural Network", "DNN_scatterplot.png")
]
for title, filename in saved_plots:
    img = plt.imread(filename)
    plt.figure(figsize=(6, 4))
    plt.imshow(img, aspect="auto") # img fill plot space
    plt.axis("off")
    plt.title(title, pad=10)       # title padding
    plt.show()

: 

Our *Gradient Boosting* model has the most dense clustering around the ideal line, indicating that it performed the best at predicting Airbnb prices out of the three models.

Let's look at our *evaluation metrics* MSE and R² now.

In [ ]:
# Compare Evaluation Metrics
eval_metrics = {
    "Multi-Linear Regression": [round(MLR_mse, 3), round(MLR_r2, 4), round(MLR_adjusted_r2, 4)],
    "Gradient Boosting": [round(GB_mse, 3), round(GB_r2, 4),round(GB_adjusted_r2, 4)],
    "Deep Neural Networks": [round(DNN_mse, 3), round(DNN_r2, 4),round(DNN_adjusted_r2, 4)]
}
eval_metrics_df = pd.DataFrame.from_dict(eval_metrics, orient="index", columns=["MSE", "R²", "Adjusted R²"])
eval_metrics_df = eval_metrics_df.sort_values(by="MSE", ascending=True) # sort by MSE
eval_metrics_df

: 

The *Gradient Boosting* has the smallest *MSE* and largest *R²*, which indicates that this model did indeed perform the best at predicting Airbnb prices.

### ii) Graphical Analysis

Let's look more closely at the *predicted* versus *actual* price comparisons for each European city using *Gradient Boosting*.

In [ ]:
# Gradient Boosting - Actual vs Predicted by City
cities = df["city"].unique() # get unique cities
city_models = {}
city_results = {}
plot_data = {}    # store data for plotting

# Run GB Model For Each City
for city in df["city"].unique():
    city_df = df[df["city"] == city]
    X_city = city_df.drop(["price", "city"], axis=1)
    y_city = city_df["price"]

    X_train, X_test, y_train, y_test = train_test_split(X_city, y_city, test_size=0.2, random_state=42)  # split
    model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=42) # model
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test) # make predictions
    mse = np.mean((y_pred - y_test.values) ** 2) # compute MSE
    r2 = r2_score(y_test, y_pred)                # compute R^2

    # store results
    city_models[city] = model
    city_results[city] = {"MSE": round(mse, 3), "R^2": round(r2, 3)}
    plot_data[city] = {"y_test": y_test.values, "y_pred": y_pred}  # save data for plotting

# Evaluate Each City
city_results_df = pd.DataFrame(city_results).T   # cities as rows
city_results_df.reset_index(inplace=True)        # reset index
city_results_df.columns = ["City", "MSE", "R^2"] # rename cols
city_results_df

: 

In [ ]:
# Scatterplots - Actual vs Predicted by City
fig, axes = plt.subplots(5, 2, figsize=(9, 18)) # init subplots
axes = axes.flatten()                           # flatten axes array

for idx, (city, data) in enumerate(plot_data.items()):
    ax = axes[idx] # get subplot idx
    y_test = data["y_test"]
    y_pred = data["y_pred"]

    ax.scatter(y_test, y_pred, alpha=0.7, color="#84206b", label="Data Points")
    ax.plot(y_test, y_test, "r--", color="#e55c30", label="Ideal Line")
    ax.set_title(f"Prices in {city.title()}")
    ax.set_xlabel("Actual")
    ax.set_ylabel("Predicted")
    ax.legend()

fig.suptitle("Actual vs. Predicted Prices by City", fontsize=16, y=1)
plt.tight_layout()
plt.show()

: 

Based on our *scatterplot* of actual vs predicted prices for each city, our model...

- ***Performance:*** performs well across most cities while many cities *(Amsterdam, Berlin, and Rome)* demonstrate a strong fit

- ***Errors:*** show minor deviations for certain cities *(Budapest and Vienna)* based on the greater dispersion in their scatterplots

Now let's use *Gradient Boosting* to plot a *feature importance* barchart.

In [ ]:
# Barchart - Feature Importance
importance_vals = GB_model.feature_importances_     # GradientBooster model
features = GB_X_train.columns                       # feature names
norm = plt.Normalize(importance_vals.min(), importance_vals.max()) # normalize scale
temp = np.argsort(importance_vals)[::-1]            # sort by importance
indices = [i for i in temp if features[i]]          # get idx
colors = cm.inferno(norm(importance_vals[indices])) # color cmap

# plot with inferno cmap
plt.figure(figsize=(8, 6))
plt.bar(range(len(indices)), importance_vals[indices], align="center", color=colors)
plt.xticks(range(len(indices)), [features[i] for i in indices], rotation=50, ha="right") # rotate labels
plt.title("Feature Importance")
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.tight_layout()
plt.show()

: 

In [ ]:
# Most Influential Features
top_5 = [features[i] for i in indices[:5]] # extract
last_5 = [features[i] for i in indices[-5:]]

top_5_importance = [round(importance_vals[i],2) for i in indices[:5]] # importance vals
last_5_importance = [round(importance_vals[i],4) for i in indices[-5:]]

top_features = {"Feature": top_5, "Importance": top_5_importance} # dict
last_features = {"Feature": last_5, "Importance": last_5_importance}

top_features_df = pd.DataFrame(top_features) # convert to DF
last_features_df = pd.DataFrame(last_features)

print(f"Most Influential Features:\n{top_features_df}\n")
print(f"Least Influential Features:\n{last_features_df}")

: 

### iii) Insights

Based on our optimal *Gradient Boosting* results, we gained practical business insights that *hosts* and *property owners* on Airbnb can benefit from.

- ***location:*** Since more centrally located listings tend to have higher prices, hosts can emphasize location or key areas in their listing descriptions.

- ***attr_index:*** Since the availability of nearby attractions heavily impacts pricing, hosts can highlight these attractions in their listings.

- ***person_capacity:*** Since listings that accommodate more people are priced higher, hosts can consider adding more beds to increase capacity and justify higher pricing.

- ***entire_home/apt:*** Since listings that offer the entire space tend to have higher prices, hosts can adjust listings to be tailored towards guests looking to book entire spaces.

- ***bedrooms:*** Since the number of bedrooms increases prices, hosts can emphasize bedroom features or add more beds.

## Conclusion

In conclusion, our analysis demonstrates that *Gradient Boosting* is the most effective model for predicting Airbnb prices across major European cities. *Gradient Boosting* outperformed *Multi-Linear Regression* and *Deep Neural Networks* in both accuracy and interpretability. Its predictions provide valuable insights that hosts and property owners at Airbnb can use to optimize their listings and maximize revenue based on key features like *location, capacity, and amenities*

## Table Cleanup

This part is for cleaning up the Python tables that we will include in our report.

In [ ]:
# add model column because plotly does not recognize index values
eval_metrics_df['Model'] = eval_metrics_df.index
eval_metrics_df = eval_metrics_df[['Model',"MSE", "R²", "Adjusted R²"]]
eval_metrics_df

: 

In [ ]:
# I used the example code from the offical website for pandas DataFrames as a template
# I also referenced the code under Alternating Row Colors (same source)
# Source: https://plotly.com/python/table/

import plotly.graph_objects as go


# fig1 is the table comparing the MSE and R^2 scores (both adjusted and not adjusted) for all the models
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'


fig1 = go.Figure(data=[go.Table(
    header=dict(values= eval_metrics_df.columns,
                fill_color='grey',
                align='left',
                font=dict(color='white'),
                line_color='black'),
    cells=dict(values= [eval_metrics_df[col] for col in eval_metrics_df.columns],
               fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*4],
               align='left',
               font=dict(color='black'),
               line_color='black',))
])
fig1.update_layout(title = "Performance of Models")

fig1.show()

: 

In [ ]:
# making a dataframe for the gradient boosting model results
gb_test_results = pd.DataFrame(columns = ['Metric', 'Value'], data = [["Test MSE",round(GB_mse,4)],["Test R²", round(GB_r2,4)], ["Test Adjusted R²",round(GB_adjusted_r2,4)]])
gb_test_results

: 

In [ ]:
# fig2 is the results of the gradient boosting model

fig2 = go.Figure(data=[go.Table(
    header=dict(values= gb_test_results.columns,
                fill_color='grey',
                align='left',
                font=dict(color='white'),
                line_color='black'),
    cells=dict(values= [gb_test_results[col] for col in gb_test_results.columns],
               fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*4],
               align='left',
               font=dict(color='black'),
               line_color='black',))
])

fig2.update_layout(title = "Performance of Gradient Boosting Model")

fig2.show()

: 

In [ ]:
# fig3 is the table of the Most Influential Features

fig3 = go.Figure(data=[go.Table(
    header=dict(values= top_features_df.columns,
                fill_color='grey',
                align='left',
                font=dict(color='white'),
                line_color='black'),
    cells=dict(values= [top_features_df[col] for col in top_features_df.columns],
               fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*4],
               align='left',
               font=dict(color='black'),
               line_color='black',))
])

fig3.update_layout(title = "Most Influential Features")

fig3.show()

: 

In [ ]:
# fig4 is the table of the Most Influential Features

fig4 = go.Figure(data=[go.Table(
    header=dict(values= last_features_df.columns,
                fill_color='grey',
                align='left',
                font=dict(color='white'),
                line_color='black'),
    cells=dict(values= [last_features_df[col] for col in last_features_df.columns],
               fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*4],
               align='left',
               font=dict(color='black'),
               line_color='black',))
])

fig4.update_layout(title = "Least Influential Features")

fig4.show()


: 